<a href="https://colab.research.google.com/github/maitreyeejadhav/Fake-News-Detection/blob/main/model_updation_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import joblib

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Load existing data
fake_news_data = pd.read_csv('train.csv')

In [ ]:
fake_news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
fake_news_data = fake_news_data.fillna('')

In [ ]:
fake_news_data['content'] = fake_news_data['author']+' '+fake_news_data['title']


In [ ]:
# Function to preprocess text
def preprocess_text(text):
  port_stem = PorterStemmer()
  text = re.sub('[^a-zA-Z]', '', text)

  text = text.lower()
  text = text.split()
  text = [port_stem.stem(word) for word in text if word not in set(stopwords.words('english'))]
  text = ' '.join(text)
  return text

In [ ]:
# Preprocess content column
fake_news_data['content'] = fake_news_data['author'] +''+ fake_news_data['title']
fake_news_data['content'] = fake_news_data['content'].apply(preprocess_text)

In [ ]:
# Split data into features and target
features = fake_news_data['content']
target = fake_news_data['label']

In [ ]:
# Convert textual data to numerical data
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(features)

In [ ]:
# Initial model training
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.2,
random_state=2)
classifier = SVC(kernel ='linear')
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [ ]:
# Function to periodically update the model
def update_model():
  while True:
    time.sleep(5) # Sleep for 1 day
    new_data = pd.read_csv('test.csv') # Load new data
    new_data['content'] = new_data['author'] + '' + new_data['title']
    new_data['content'] = new_data['content'].apply(preprocess_text)
    new_features = vectorizer.transform(new_data['content']) # Transform new data to numerical features
    combined_features = sparse.vstack([features, new_features]) # Combine existing and new features
    combined_target = pd.concat([target, new_data['label']], ignore_index=True) # Combine existing and new target
    X_train, X_test, Y_train, Y_test = train_test_split(combined_features, combined_target, test_size=0.2, random_state=2)
    classifier.fit(X_train, Y_train) # Retrain the model
    Y_test_pred = classifier.predict(X_test) # Predict on test data
    accuracy = accuracy_score(Y_test, Y_test_pred) # Calculate accuracy
    print('Updated Model Accuracy: ', accuracy)
    if accuracy > 0.9: # Save the model if accuracy is above a certain threshold
      joblib.dump(classifier, 'fake_news_classifier.pkl')
      print('Model saved')




In [ ]:
# Start updating the model periodically
update_model()

TypeError: expected string or bytes-like object